In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline



hf_pipeline = pipeline(
    "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    max_new_tokens=150,
    device_map="auto",
    temperature=0.0,
    do_sample=False,
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)


def calculator_tool(input_text):
    try:
        expression = input_text.replace("calculate", "").strip()
        return str(eval(expression))
    except:
        return "Error in calculation"

def search_tool(query):
    return f"Search results for: {query}"



class AgentState(TypedDict):
    input: str
    action: str
    tool_result: str
    output: str



def planner(state: AgentState):
    if "calculate" in state["input"]:
        return {"action": "calculator"}
    return {"action": "search"}

def tool_executor(state: AgentState):
    if state["action"] == "calculator":
        return {"tool_result": calculator_tool(state["input"])}
    return {"tool_result": search_tool(state["input"])}

def final_response(state: AgentState):
    if state["action"] == "calculator":
        return {"output": state["tool_result"]}
    else:
        return {"output": llm.invoke(state["input"])}



graph = StateGraph(AgentState)

graph.add_node("planner", planner)
graph.add_node("tool", tool_executor)
graph.add_node("final", final_response)

graph.set_entry_point("planner")
graph.add_edge("planner", "tool")
graph.add_edge("tool", "final")

app = graph.compile()


response = app.invoke({"input": "calculate 3553*6"})
print(response["output"])


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

21318
